In [3]:
from openai.embeddings_utils import get_embedding

In [7]:
engine = 'text-embedding-ada-002'

In [9]:
len(get_embedding('text', engine=engine))

1536

In [10]:
import pandas as pd

In [11]:
df_train = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_train.json', 
    lines=True)
df_val = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_val.json', 
    lines=True)
df_test = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_test.json', 
    lines=True)

In [13]:
from tqdm import tqdm

In [14]:
recs = []
for i in tqdm(df_test.to_dict('records')):
    i['openai_embedding'] = get_embedding(i['query'], engine=engine)
    recs.append(i)

df_test_emb = pd.DataFrame(recs)

100%|██████████| 8840/8840 [18:39<00:00,  7.89it/s]  


In [16]:
recs = []
for i in tqdm(df_val.to_dict('records')):
    i['openai_embedding'] = get_embedding(i['query'], engine=engine)
    recs.append(i)

df_val_emb = pd.DataFrame(recs)

100%|██████████| 8839/8839 [16:27<00:00,  8.95it/s]  


In [17]:
recs = []
for i in tqdm(df_train.to_dict('records')):
    i['openai_embedding'] = get_embedding(i['query'], engine=engine)
    recs.append(i)

df_train_emb = pd.DataFrame(recs)

100%|██████████| 26518/26518 [1:04:01<00:00,  6.90it/s]  


In [18]:
df_train_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_train_oaiemb.json', 
    lines=True, orient='records')
df_val_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_val_oaiemb.json', 
    lines=True, orient='records')
df_test_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_test_oaiemb.json', 
    lines=True, orient='records')

In [19]:
len(df_train_emb), len(df_val_emb), len(df_test_emb)

(26518, 8839, 8840)